# 1. Librerías

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
from sklearn.ensemble import IsolationForest
from sklearn import metrics
import shap
#from utils.Validator import *
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

c:\GIT_CUMP4\epic\.venv\lib\site-packages\shap\utils\_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
c:\GIT_CUMP4\epic\.venv\lib\site-packages\shap\utils\_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_minimization_order(all_masks, max_swap_size=100, 

# 2. Data

In [2]:
dat= pd.read_csv(r"../data/modeling/03_train_ppnn.csv",sep="|")
dat.head()

,PERIODO,CODCLAVECIC,EDAD,TIPPER,ANTIGUEDAD,CODACTECONOMICA,DESACTECONOMICA,FLG_ACTECO_NODEF,FLGNP,CTDNP,FLGLSB,CTDLSB,FLGARCHIVONEGATIVO,DESTIPMOTIVONEGATIVO,CTD_NP_LSB,CTDEVAL,CODSUBSEGMENTO,DESSUBSEGMENTO,CODSEGMENTO,DESSEGMENTO,TIPDIR,CODUBIGEO,CODDISTRITO,DESCODDISTRITO,CODPROVINCIA,DESCODPROVINCIA,CODDEPARTAMENTO,DESCODDEPARTAMENTO,CODPAISNACIONALIDAD,DESCODPAISNACIONALIDAD,FLGNACIONALIDAD,MTO_INGRESOS_MES,CTD_INGRESOS_MES,MEDIA_INGRESOS,DESV_INGRESOS,FLG_PERFIL_INGRESOS_3DS,MAX_CTDMTOSREDONDOS,SUM_MTOSREDONDOS,CTDMAXIMA,MTO_MAXIMOSREPETIDOS,MTOMAXDEMAXREPETIDOS,MTO_CONOTROSPROXIMOS,CTD_CONOTROSPROXIMOS
0,202309,15122500,34,P,93,2213,EDICION DE GRABACIONES DE DISCOS Y OTROS,0,0,0,0,0,0,NaN,0,0,X1N,EXCLUSIVO NORMAL,PX,EXCLUSIVO,D,150104,1299,BARRANCO-LIMA,M7,LIMA,30.0000,LIMA,PER,PERU,0,11200.0000,1,10000.0000,0.0000,0,1,11200.0000,1,11200.0000,11200.0000,11200.0000,1
1,202309,6530808,57,P,187,0091,EMPLEADOS,0,0,0,0,0,0,NaN,0,0,M1N,CONSUMO NORMAL,PM,CONSUMO,D,010101,0023,CHACHAPOYAS-AMAZONAS,A3,CHACHAPOYAS,2.0000,AMAZONAS,PER,PERU,0,270.0000,2,195.0000,7.0700,1,1,270.0000,1,150.0000,150.0000,150.0000,1
2,202309,27384645,19,P,8,0091,EMPLEADOS,0,0,0,0,0,0,NaN,0,0,M1N,CONSUMO NORMAL,PM,CONSUMO,D,060803,0605,CHONTALI-CAJAMARCA,F9,JAEN,12.0000,CAJAMARCA,PER,PERU,0,150.4900,1,750.0000,494.9700,0,0,0.0000,1,150.4900,150.4900,150.4900,1
3,202309,14088498,61,P,103,0099,INFORMACION NO DISPONIBLE,1,0,0,0,0,0,NaN,0,0,M1N,CONSUMO NORMAL,PM,CONSUMO,D,130104,1165,HUANCHACO-LA LIBERT,L6,TRUJILLO,26.0000,LA LIBERTAD,PER,PERU,0,310.0000,2,456.6700,177.8600,0,1,310.0000,1,200.0000,200.0000,200.0000,1
4,202309,10448617,35,P,114,0099,INFORMACION NO DISPONIBLE,1,0,0,0,0,0,NaN,0,0,M1N,CONSUMO NORMAL,PM,CONSUMO,D,150106,1301,CARABAYLLO-LIMA,M7,LIMA,30.0000,LIMA,PER,PERU,0,900.4900,3,150.4900,0.0000,0,1,750.0000,1,400.0000,400.0000,400.0000,1


# 3. Escalón 2 - Población con riesgo

In [3]:
esc2=dat[(dat.MTO_INGRESOS_MES>=150)&(dat.MTO_INGRESOS_MES<=1e+6)]

# 4. Escalón 3 - Directo a evaluación

In [4]:
esc3=dat[dat.MTO_INGRESOS_MES>1e+6]

# 5. Modelamiento

## Modelo 1: k-MEANS

#### Con las variables originales

In [5]:
x_model=['MTO_CONOTROSPROXIMOS', 'CTD_CONOTROSPROXIMOS']

In [6]:
# Se tomará una muestra del 30% ppor periodo
from sklearn.model_selection import train_test_split
muestra1, muestra2 = train_test_split(esc2, train_size=0.7, stratify=esc2['PERIODO'])

In [7]:
train_nat_std = StandardScaler().fit_transform(muestra2[x_model])

k = [3,4,5,6,7,8,9,10,11,12,13,14,15]
for ki in k:
        outputKMeans = KMeans(n_clusters = ki, init='k-means++', random_state = 101).fit(train_nat_std)
        scoreSilhoutte = metrics.silhouette_score(train_nat_std, outputKMeans.labels_, metric='euclidean')  
        print(" k=", ki," SC=", scoreSilhoutte)

 k= 3  SC= 0.9312888142810433
 k= 4  SC= 0.9318500514554385
 k= 5  SC= 0.9363762863815998
 k= 6  SC= 0.9395391247279105
 k= 7  SC= 0.9441855353533034
 k= 8  SC= 0.9310521500584368
 k= 9  SC= 0.9446269559973457
 k= 10  SC= 0.9264108130207536
 k= 11  SC= 0.9302387683875528
 k= 12  SC= 0.8813141624872237
 k= 13  SC= 0.8802442154723218
 k= 14  SC= 0.881894832456393
 k= 15  SC= 0.8638837901562753


#### Agregando FLG_PERFIL_INGRESOS_3DS

In [8]:
x_model2=['MTO_CONOTROSPROXIMOS', 'CTD_CONOTROSPROXIMOS','FLG_PERFIL_INGRESOS_3DS']

In [9]:
train_nat_std = StandardScaler().fit_transform(muestra2[x_model2])

k = [3,4,5,6,7,8,9,10]
for ki in k:
        outputKMeans = KMeans(n_clusters = ki, init='k-means++', random_state = 101).fit(train_nat_std)
        scoreSilhoutte = metrics.silhouette_score(train_nat_std, outputKMeans.labels_, metric='euclidean')  
        print(" k=", ki," SC=", scoreSilhoutte)

 k= 3  SC= 0.892181994210693
 k= 4  SC= 0.9245177567720616
 k= 5  SC= 0.9252378989858179
 k= 6  SC= 0.929251200181359
 k= 7  SC= 0.9304087934729427
 k= 8  SC= 0.935516982635988
 k= 9  SC= 0.9420485907047218
 k= 10  SC= 0.933518073990612


## Modelo 2: Isolation Forest

#### Con las variables originales

In [10]:
silhouette_list = []

for cont in np.arange(0.0001,0.0005,0.0001):
    cluster_labels = IsolationForest(n_estimators = 200,
                                    n_jobs=-1,
                                    max_features=(len(muestra2[x_model].columns)),
                                    contamination=cont,#proporcion de outliers en el dataset
                                    random_state=66,bootstrap=True).fit_predict(muestra2[x_model])
    
    out_n = cluster_labels[cluster_labels==-1].shape[0]
    sil = silhouette_score(muestra2[x_model],cluster_labels, metric='euclidean')
    silhouette_list.append(sil)
    print("silhouette for "+str(np.round(cont,4))+" with n=" +str(out_n)+" outliers is: "+str(sil))

silhouette for 0.0001 with n=8 outliers is: 0.9755886466752463
silhouette for 0.0002 with n=9 outliers is: 0.9747311985178461
silhouette for 0.0003 with n=9 outliers is: 0.9747311985178461
silhouette for 0.0004 with n=31 outliers is: 0.9897447261975761


#### Agregando FLG_PERFIL_INGRESOS_3DS

In [11]:
x_model2=['MTO_CONOTROSPROXIMOS', 'CTD_CONOTROSPROXIMOS','FLG_PERFIL_INGRESOS_3DS']

In [12]:
silhouette_list = []

for cont in np.arange(0.0001,0.0006,0.0001):
    cluster_labels = IsolationForest(n_estimators = 200,
                                    n_jobs=-1,
                                    max_features=(len(muestra2[x_model2].columns)),
                                    contamination=cont,#proporcion de outliers en el dataset
                                    random_state=66,bootstrap=True).fit_predict(muestra2[x_model2])
    
    out_n = cluster_labels[cluster_labels==-1].shape[0]
    sil = silhouette_score(muestra2[x_model2],cluster_labels, metric='euclidean')
    silhouette_list.append(sil)
    print("silhouette for "+str(np.round(cont,4))+" with n=" +str(out_n)+" outliers is: "+str(sil))

silhouette for 0.0001 with n=7 outliers is: 0.9754225282374555
silhouette for 0.0002 with n=16 outliers is: 0.9670789681235955
silhouette for 0.0003 with n=21 outliers is: 0.9721464914470905
silhouette for 0.0004 with n=31 outliers is: 0.974828270430136
silhouette for 0.0005 with n=39 outliers is: 0.9734018269675678


# Modelo Final

In [13]:
X_train = esc2[x_model2]

In [14]:
iforest = IsolationForest(n_estimators = 200,
                                    n_jobs=-1,
                                    max_features=(len(X_train[x_model2].columns)),
                                    contamination=0.0004,
                                    random_state=66,bootstrap=True).fit(X_train[x_model2])

In [15]:
esc2['if_ros']=iforest.predict(esc2[x_model2])

In [16]:
esc2[esc2.if_ros==-1].PERIODO.value_counts().mean()

17.0

In [17]:
esc2[esc2.if_ros==-1].describe().T

,count,mean,std,min,25%,50%,75%,max
PERIODO,102.0000,202306.6667,1.6431,202304.0000,202305.0000,202307.0000,202308.0000,202309.0000
CODCLAVECIC,102.0000,5109788.1176,6796297.7167,9941.0000,296640.7500,2541913.0000,6524377.2500,26990738.0000
EDAD,102.0000,54.1373,14.2002,24.0000,46.0000,53.5000,65.0000,84.0000
ANTIGUEDAD,102.0000,257.3725,129.8517,1.0000,152.2500,269.5000,341.0000,632.0000
FLG_ACTECO_NODEF,102.0000,0.4510,0.5000,0.0000,0.0000,0.0000,1.0000,1.0000
FLGNP,102.0000,0.0294,0.1698,0.0000,0.0000,0.0000,0.0000,1.0000
CTDNP,102.0000,0.1176,0.8478,0.0000,0.0000,0.0000,0.0000,8.0000
FLGLSB,102.0000,0.0392,0.1951,0.0000,0.0000,0.0000,0.0000,1.0000
CTDLSB,102.0000,0.2353,1.5167,0.0000,0.0000,0.0000,0.0000,13.0000
FLGARCHIVONEGATIVO,102.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [18]:
#Guardamos el modelo
import pickle
pickle.dump(iforest, open(r"../src/02_models/iforest_model_ppnn", 'wb'))